In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2023-03-12 17:03:06.592838: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 17:03:07.152141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib/:/opt/cuda/targets/x86_64-linux/lib/:/opt/cuda/nvvm/libdevice/:/home/f14/Downloads/cudnn-linux-x86_64-8.8.1.3_cuda11-archive/lib/:/opt/cuda/nvvm/libdevice/
2023-03-12 17:03:07.152196: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2023-03-12 17:03:07.789234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 17:03:07.789336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import wget, zipfile, os, random

# download dataset
if not os.path.isfile("data.zip"): 
    print("downloading...")
    wget.download("https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/3362/31148/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1678299698&Signature=b6XuKNXvsSuBcv2SZYiTqcfr7Mc%2FjpSR%2BptWRsmjRTHr1vqLdnpC6k9YaLk%2BMRSmTeZcOXGFg21TGUyv9RR7TctRcWLW8WhIdibLM2BTLqc2YCM5ZXU9DkwYRypzgmXjZs4U%2B2n4AeOzO2w7CJFnxgjOeOtAamziNd5F07aiSap1A8b2PwtClwvsQA0kx1vp0HTtT99NsWpGFY310ZOMjRnpFK17ANh5UmYAaFeHm2RVyR%2FRRVcWbVTSWKhaEDLgppgdIREZ%2FGsXpybkFAqfWGqmHkDVEpty0E3BuyGGZ8DEkDphwjHlab824Waid0IpOeOgO6qAziK5fy22dCO0Pg%3D%3D&response-content-disposition=attachment%3B+filename%3Ddogs-vs-cats.zip", "data.zip")

# extract dataset
if not (os.path.isdir("data") and os.path.isdir("data/train") and os.path.isdir("data/test1")): 
    print("extracting...")
    with zipfile.ZipFile("data.zip"      , 'r') as file: file.extractall("./data/")
    with zipfile.ZipFile("data/train.zip", 'r') as file: file.extractall("./data/")
    with zipfile.ZipFile("data/test1.zip", 'r') as file: file.extractall("./data/")

# get all train paths
paths = list(map(lambda name:os.path.join("data/train",name), os.listdir("data/train/")))
random.shuffle(paths)
test_paths  = paths[:len(paths)//3]
train_paths = paths[len(paths)//3+1:]

In [18]:
import random, copy
def batchfy(paths, batch_size):
    #shuffle all paths
    paths = copy.deepcopy(paths)
    random.shuffle(paths)
    
    # batchfy
    size = len(paths)
    batches = size // batch_size
    batches = [paths[i*batch_size:(i+1)*batch_size] for i in range(batches)]
    
    return batches


def load_batch(paths):
    tensor_images = tf.stack([tf.image.resize(tf.io.decode_image(tf.io.read_file(path),channels=3), (256,256))/256 for path in paths])
    tensor_labels = tf.convert_to_tensor([[1,0] if path[11:14] == "cat" else [0,1] for path in paths], dtype=float)
    return tensor_images, tensor_labels


In [53]:

class ConvBlock(tf.keras.Model):

    def __init__(self, kernel_size=(3,3), filters=16, strides=(2, 2)):
        super().__init__()

        self.conv2a = tf.keras.layers.Conv2D(filters, (1, 1), strides=strides, )
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

        self.convd = tf.keras.layers.Conv2D(filters, (1, 1), strides=strides)
        self.bnd = tf.keras.layers.BatchNormalization()

    def call(self, z):
        x = tf.nn.relu(self.bn2a(self.conv2a(z)))
        x = tf.nn.relu(self.bn2b(self.conv2b(x)))
        x = self.bn2c(self.conv2c(x))

        z = self.convd(z)
        z = self.bnd(z)

        return tf.nn.relu(x + z)


In [60]:
class MyResNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2))
        self.bn_conv1 = tf.keras.layers.BatchNormalization()
        self.max_pool = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))

        self.c0 = ConvBlock()
        self.c1 = ConvBlock()
        self.c2 = ConvBlock()
        self.c3 = ConvBlock()
        self.c4 = ConvBlock()
        
        # GAP
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.lin = tf.keras.layers.Dense(2, activation="softmax")
        
    def call(self, x):
      
        x = self.conv1(x)
        x = self.bn_conv1(x)
        x = tf.nn.relu(x)
        x = self.max_pool(x)
        
        x = self.c0(x)
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)

        x = self.lin(self.gap(x))
        
        return x

In [63]:
batch_size = 64
epochs = 100
model = MyResNet()
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy(axis=-1)

for e in range(epochs):
    cma = 0
    
    # shuffle and batch the dataset
    batched_paths = batchfy(paths, batch_size)
    
    for i, batch in enumerate(batched_paths):
        X, Y = load_batch(batch)
        
        with tf.GradientTape() as tape:
            P = model(X)
            L = loss(Y, P)
        
        A = tf.reduce_mean(tf.cast(tf.argmax(P,-1) == tf.argmax(Y,-1), float))
        cma = cma + (A-cma) / (i+1)
        G = tape.gradient(L, model.trainable_variables)
        optim.apply_gradients(zip(G, model.trainable_variables))

        print(f"\r epoch:{e}, batch:{i}/{len(batched_paths)}, loss: {L.numpy():.5f}, acc: {A.numpy():.5f}, cma: {cma:.5f}", end=" "*5)
    print()


 epoch:0, batch:389/390, loss: 0.66907, acc: 0.53125, cma: 0.55950     
 epoch:1, batch:389/390, loss: 0.65776, acc: 0.59375, cma: 0.60076     
 epoch:2, batch:389/390, loss: 0.61214, acc: 0.62500, cma: 0.63353     
 epoch:3, batch:389/390, loss: 0.51184, acc: 0.78125, cma: 0.67588     
 epoch:4, batch:389/390, loss: 0.50566, acc: 0.76562, cma: 0.69820     
 epoch:5, batch:389/390, loss: 0.63617, acc: 0.64062, cma: 0.70889     
 epoch:6, batch:389/390, loss: 0.53048, acc: 0.71875, cma: 0.71807     
 epoch:7, batch:389/390, loss: 0.70511, acc: 0.60938, cma: 0.72432     
 epoch:8, batch:389/390, loss: 0.53825, acc: 0.70312, cma: 0.72901     
 epoch:9, batch:389/390, loss: 0.48911, acc: 0.81250, cma: 0.73914     
 epoch:10, batch:389/390, loss: 0.40406, acc: 0.84375, cma: 0.74359     
 epoch:11, batch:389/390, loss: 0.42890, acc: 0.82812, cma: 0.74587     
 epoch:12, batch:389/390, loss: 0.44459, acc: 0.87500, cma: 0.75224     
 epoch:13, batch:389/390, loss: 0.44304, acc: 0.81250, cma: 0